In [22]:
import torch, os, sys, fnmatch, argparse
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
sys.path.insert(1,'../')
import utils


GeV = 1/1000

dataset = "dataset_2_padded_nentry"
preproc_dataset_name = 'ds2_diff_transforms'
dataset_store_path = os.path.join("/eos/user/j/jthomasw/tdsm_encoder/datasets/", preproc_dataset_name)
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print('Device: ', device)

dataset_max_dist_E = 0.

# Input files
files_list_ = []
for filename in os.listdir(dataset_store_path):
    if fnmatch.fnmatch(filename, dataset + '*.pt'):
        file_path = os.path.join(dataset_store_path, filename)
        print(f'file_path: {file_path}')
        loaded_file = torch.load(file_path)
        showers_ = loaded_file[0]
        incident_e = loaded_file[1]*GeV
        custom_data = utils.cloud_dataset(file_path, device=device)
        for i, sample_i in enumerate(custom_data.data):

            if i >= len(custom_data.data)-1: 
                break

            shower1 = custom_data.data[i]
            shower2 = custom_data.data[i+1]
            
            
            E_1 = torch.mul(shower1[:,0],GeV)
            shower1 = torch.stack((E_1, shower1[:,1], shower1[:,2], shower1[:,3]), -1)
            
            for j, sample_j in enumerate(custom_data.data):
                if j == i:
                    continue
                E_2 = torch.mul(shower2[:,0],GeV)
                shower2 = torch.stack((E_2, shower2[:,1], shower2[:,2], shower2[:,3]), -1)

                # batched distance between each pair of the two collections
                #output = torch.cdist(shower1, shower2)
                cdist_E = torch.cdist(E_1, E_2)
                cdist_X = torch.cdist(shower1[:,1], shower2[:,1])
                cdist_Y = torch.cdist(shower1[:,2], shower2[:,2])
                cdist_Z = torch.cdist(shower1[:,3], shower1[:,3])

                max_dist_E = torch.max(cdist_E)
                max_dist_X = torch.max(cdist_X)
                max_dist_Y = torch.max(cdist_Y)
                max_dist_Z = torch.max(cdist_Z)
                if max_dist_E > dataset_max_dist_E:
                    print(f'max_dist_E {max_dist_E}: ')
                    dataset_max_dist_E = max_dist_E

#print(f'Dataset pairwise max distance: {dataset_max_dist}')


Device:  cpu
file_path: /eos/user/j/jthomasw/tdsm_encoder/datasets/ds2_diff_transforms/dataset_2_padded_nentry1129To1269.pt


RuntimeError: cdist only supports at least 2D tensors, X1 got: 1D